In [1]:
import os

os.chdir("..")
os.chdir("D:\du-ws\\text2SQL\chatbot_medical_retriever")


In [5]:
import nest_asyncio
nest_asyncio.apply()
import VinRAG
import importlib
#using importlib to reload module
importlib.reload(VinRAG)
from VinRAG import VinmecRetriever
import streamlit as st

DB_VECTOR = "storage_index"
DB_ROOT = "api"
url_pg_vector = st.secrets['url_pg_vector'] +"/{db}" #store information embedding

model = "gpt-3.5-turbo-1106"
api_key = st.secrets['api_key']
vin_retriever = VinmecRetriever(DB_VECTOR, DB_ROOT, url_pg_vector,model,api_key)

connect successfully


In [6]:
question = "Triệu chứng kiệt sức"
title,title_str,list_link = vin_retriever.get_customRV(question)
# title
# ans = vin_retriever.get_customRV(question)

In [26]:
title

[('https://www.vinmec.com/tin-tuc/thong-tin-suc-khoe/suc-khoe-tong-quat/cac-hoi-chung-benh-ly-ho-hap-thuong-gap/',
  'Các hội chứng, bệnh lý hô hấp thường gặp,  Tràn dịch màng phổi,  Tràn khí màng phổi,  Hội chứng đông đặc phổi,  Hội chứng hạn chế và hội chứng tắc nghẽn,'),
 ('https://www.vinmec.com/tin-tuc/thong-tin-suc-khoe/suc-khoe-tong-quat/trieu-chung-lam-sang-va-hinh-anh-hep-ong-song-do-truot-dot-song-lung/',
  'Triệu chứng lâm sàng và hình ảnh hẹp ống sống do trượt đốt sống thắt lưng,  Hẹp ống sống do trượt đốt sống thắt lưng là gì?,  Triệu chứng lâm sàng của hẹp ống sống do trượt đốt sống thắt lưng,  Hình ảnh hẹp ống sống do trượt đốt sống thắt lưng,'),
 ('https://www.vinmec.com/tin-tuc/thong-tin-suc-khoe/dinh-duong/trieu-chung-say-san-ngo-doc-san/',
  'Triệu chứng say sắn, ngộ độc sắn,  Biểu hiện ngộ độc sắn,  Vì sao ăn sắn có thể bị ngộ độc,'),
 ('https://www.vinmec.com/tin-tuc/thong-tin-suc-khoe/suc-khoe-tong-quat/hoi-chung-met-moi-ke-gay-hau-qua-tham-lang/',
  'Hội chứng mệ

In [15]:
ans = vin_retriever.decide_index_retriever(question,title_str)
ans


        Bạn là người trợ giúp hữu ích trong việc giúp xác định các mục trong danh sách dưới đây tương ứng với cùng loại bệnh như trong truy vấn của tôi.
        Hãy chắc chắn rằng bạn đọc tất cả chúng một cách cẩn thận.
        Vui lòng xem xét danh sách các mô tả bệnh tật và chỉ ra các mục mô tả cùng loại bệnh như trong câu hỏi của tôi.
        Chỉ mục được chọn bao gồm các mục mô tả các triệu chứng hoặc vấn đề liên quan đến cùng loại bệnh như được đề cập trong truy vấn. Xác định tất cả các chỉ số có liên quan.
        Không cần phải giải thích.
        Luôn trả lời số chỉ mục.
        Ví dụ:
        Hỏi: Chăm sóc và điều trị bong gân cổ?
        Thông tin:
        1. Bị bong gân cổ phải làm sao?
        2. Trẻ mới biết đi từ 2-4 tuổi có bị trầm cảm không?
        3. Hướng dẫn chăm sóc trẻ bị ho
        4. Trẻ rất nhạy cảm
        5. Làm thế nào để giảm căng cơ cổ khi ngủ?
        6. Điều trị và chăm sóc trẻ nhỏ bị viêm phổi
        Chỉ số đã chọn: 1, 5
        Nếu không có chỉ mục t

'4, 5'

In [16]:
link_selected = vin_retriever.get_index(ans,title)
link_selected

['https://www.vinmec.com/tin-tuc/thong-tin-suc-khoe/suc-khoe-tong-quat/hoi-chung-met-moi-ke-gay-hau-qua-tham-lang/',
 'https://www.vinmec.com/tin-tuc/thong-tin-suc-khoe/suc-khoe-tong-quat/met-moi-co-phai-la-benh/']

In [17]:
excluse_node = vin_retriever.get_excluse_node(link_selected)
len(excluse_node)

2

In [43]:
vin_retriever.conn.rollback()

In [22]:
selected_node = []
with vin_retriever.conn.cursor() as cur:
    for node in excluse_node:
        query = "SELECT text,metadata_,node_id FROM data_vinmec_storage_index  WHERE node_id   = '{}'".format(node['id_'])
        cur.execute(query)
        dat = cur.fetchone()
        if dat:
            selected_node.append(dat)
len(selected_node)

1

In [23]:
selected_node

[('Cho đến hiện nay, không có bộ xét nghiệm tiêu chuẩn nào giúp bác sĩ chẩn đoán bệnh. Khi không có xét nghiệm hoặc dấu hiệu sinh học cụ thể về bệnh, các bác sĩ sẽ không có lựa chọn nào khác ngoài việc chẩn đoán dựa trên các triệu chứng mà bệnh nhân khai báo. Một số triệu chứng được coi là bắt buộc để chẩn đoán, trong khi những triệu chứng khác được coi là hỗ trợ. Việc chẩn đoán ở người lớn còn khá miễn cưỡng cho đến khi họ bị bệnh ít nhất 6 tháng và các bệnh đồng mắc của họ đã được giải quyết. Đối với trẻ em và thanh thiếu niên, thời gian mắc bệnh đã được rút ngắn xuống còn 3 tháng, nhưng bác sĩ điều trị tùy theo tình trạng cụ thể mà quyết định.Từ những mô tả trên, chúng ta dễ dàng nhận ra rằng việc điều trị bệnh cũng gặp nhiều khó khăn. Công tác điều trị sẽ liên quan đến nhiều chuyên khoa và cần có sự phối hợp đồng bộ. Cho đến nay chưa có một phác đồ điều trị nhất quán. Bất kỳ phương pháp điều trị nào được sử dụng để giảm bớt các triệu chứng đều được coi là "giảm nhẹ".Hội chứng mệt m

In [25]:
excluse_node_ = [node for node in excluse_node if node['id_'] not in [node[2] for node in selected_node if node is not None]]
print(len(excluse_node_))
excluse_node_

1


[{'id_': 'ca296b7a-0546-4bb2-9b63-d701cbb6fa24',
  'embedding': None,
  'metadata': {'url': 'https://www.vinmec.com/tin-tuc/thong-tin-suc-khoe/suc-khoe-tong-quat/hoi-chung-met-moi-ke-gay-hau-qua-tham-lang/'},
  'excluded_embed_metadata_keys': [],
  'excluded_llm_metadata_keys': [],
  'relationships': {<NodeRelationship.SOURCE: '1'>: {'node_id': '69a6e8ab-054f-4251-9d5e-b340d2fd584c',
    'node_type': <ObjectType.DOCUMENT: '4'>,
    'metadata': {'url': 'https://www.vinmec.com/tin-tuc/thong-tin-suc-khoe/suc-khoe-tong-quat/hoi-chung-met-moi-ke-gay-hau-qua-tham-lang/'},
    'hash': 'c4fdfdd32690abc82e764fc3f1c7abc822c29e4b05ee5fb9ba36ec8435065749',
    'class_name': 'RelatedNodeInfo'},
   <NodeRelationship.PREVIOUS: '2'>: {'node_id': 'b1d44e2c-ef4f-4351-bcee-eaa12e1db222',
    'node_type': <ObjectType.TEXT: '1'>,
    'metadata': {'url': 'https://www.vinmec.com/tin-tuc/thong-tin-suc-khoe/lam-dep/tai-sao-moi-ban-bi-sung-khi-vua-thuc-day/'},
    'hash': '51aca7f7bca1b3682d4c0bb04170a7a0eff954

In [103]:
selected_node[4]

IndexError: list index out of range

In [88]:
# excluse_node = [node for node in excluse_node if node['id_'] not in [node[2] for node in selected_node]]
# excluse_node
excluse_node_ = []
for node in excluse_node:
    for node_ in selected_node:
        if node['id_'] == node_[2]:
            excluse_node_.append(node)
            break

excluse_node_
            
            

TypeError: 'NoneType' object is not subscriptable

In [10]:
import psycopg2
from llama_index import VectorStoreIndex, Document
import requests
from bs4 import BeautifulSoup
from llama_index import VectorStoreIndex, ServiceContext, Document,StorageContext
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.postprocessor import SimilarityPostprocessor
from llama_index.memory import ChatMemoryBuffer
from llama_index.llms import OpenAI
import asyncio
import streamlit as st
from sqlalchemy import make_url
from llama_index.vector_stores import PGVectorStore
from llama_index import load_index_from_storage
from llama_index.indices.vector_store.retrievers import VectorIndexRetriever
from llama_index.query_engine.retriever_query_engine import (
    RetrieverQueryEngine,
)
from llama_index import get_response_synthesizer
from llama_index.prompts import PromptTemplate

from llama_index.indices.postprocessor import SimilarityPostprocessor
from llama_index.indices.postprocessor import LLMRerank
from llama_index.indices.postprocessor import SentenceEmbeddingOptimizer
import json
from CustomRV import CustomRetriever
from llama_index.schema import NodeWithScore,TextNode,NodeRelationship,ObjectType,RelatedNodeInfo

In [27]:
storage_instance = StorageContext.from_defaults(docstore=vin_retriever.index.docstore)
instance_vectorindex = VectorStoreIndex(excluse_node,storage_context=storage_instance)
instance_search = VectorIndexRetriever(instance_vectorindex,similarity_top_k=3)
instance_ans = instance_search.retrieve(question) #find node high score in excluse node
selected_node_with_score = [node for node in vin_retriever.ans if node.node.node_id in [_[2] for _ in vin_retriever.selected_node]]
combine_node = [node.node for node in selected_node_with_score + instance_ans]
combine_node

[TextNode(id_='8bb3ec62-f886-49a5-ae3f-9c1a46d9ecad', embedding=None, metadata={'url': 'https://www.vinmec.com/tin-tuc/thong-tin-suc-khoe/suc-khoe-tong-quat/huong-dan-xu-tri-cap-cuu-hoc-nghen/'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='5d1ab158-fc05-436a-82b1-bbc9bd0a3e86', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'url': 'https://www.vinmec.com/tin-tuc/thong-tin-suc-khoe/suc-khoe-tong-quat/huong-dan-xu-tri-cap-cuu-hoc-nghen/'}, hash='cef7ac877debaa978ef7b44aa026694b69e2134b3d70c66593c1eac098238b6c'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='c0ca0dbc-2d84-45b8-83e1-581691a66efd', node_type=<ObjectType.TEXT: '1'>, metadata={'url': 'https://www.vinmec.com/tieu-hoa-gan-mat/thong-tin-suc-khoe/bi-gan-nhiem-mo-uong-gi-het/'}, hash='a4003e62884b12066fc5e60ea358266fe179c0ec5685ba0ace36cafee1c472e8'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='2d6ae4f4-c6db-4253-b8

In [16]:
instance_vectorindex = VectorStoreIndex(combine_node,storage_context=storage_instance)
query_engine = RetrieverQueryEngine(
    retriever=VectorIndexRetriever(instance_vectorindex,similarity_top_k=3),response_synthesizer=get_response_synthesizer(response_mode="tree_summarize",streaming=True)
    )

query_engine = vin_retriever.prompt_format(query_engine)
response = query_engine.query(question)
for source in response.source_nodes:
    print(source.metadata['url'])
for text in response.response_gen:
    print(text,end='')

https://www.vinmec.com/tin-tuc/thong-tin-suc-khoe/suc-khoe-tong-quat/huong-dan-xu-tri-cap-cuu-hoc-nghen/
https://www.vinmec.com/tin-tuc/thong-tin-suc-khoe/nhi/so-cuu-tre-bi-hoc-di-vat-gay-nghet-tho/
https://www.vinmec.com/tin-tuc/thong-tin-suc-khoe/suc-khoe-tong-quat/huong-dan-xu-tri-cap-cuu-hoc-nghen/
Khi bị hóc nghẹn ở cổ, bạn cần hành động ngay lập tức để giúp nạn nhân. Đầu tiên, hỏi ngay nạn nhân xem anh/chị có bị hóc không và có cần giúp đỡ không. Nếu nạn nhân gật đầu đồng ý, bạn cần can thiệp ngay.

Đứng hoặc quỳ phía sau nạn nhân (tùy vào kích thước của nạn nhân), có thể bước một chân trụ tới trước để tư thế vững vàng hơn. Vòng hai tay qua, nắm hai bàn tay ở phía trước nạn nhân. Một bàn tay nắm lại tạo thành một nắm đấm. Đặt mặt có ngón cái của nắm đấm vào giữa vị trí trên rốn và dưới xương ức. Bàn tay kia nắm lấy nắm đấm, thúc nhanh và mạnh vào bụng nạn nhân về phía sau và lên trên. Tiếp tục thực hiện nhiều lần ép mạnh như vậy cho đến khi dị vật văng ra và nạn nhân có thể thở, 

In [63]:
combine_node

[NodeWithScore(node=TextNode(id_='8bb3ec62-f886-49a5-ae3f-9c1a46d9ecad', embedding=None, metadata={'url': 'https://www.vinmec.com/tin-tuc/thong-tin-suc-khoe/suc-khoe-tong-quat/huong-dan-xu-tri-cap-cuu-hoc-nghen/'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='5d1ab158-fc05-436a-82b1-bbc9bd0a3e86', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'url': 'https://www.vinmec.com/tin-tuc/thong-tin-suc-khoe/suc-khoe-tong-quat/huong-dan-xu-tri-cap-cuu-hoc-nghen/'}, hash='cef7ac877debaa978ef7b44aa026694b69e2134b3d70c66593c1eac098238b6c'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='c0ca0dbc-2d84-45b8-83e1-581691a66efd', node_type=<ObjectType.TEXT: '1'>, metadata={'url': 'https://www.vinmec.com/tieu-hoa-gan-mat/thong-tin-suc-khoe/bi-gan-nhiem-mo-uong-gi-het/'}, hash='a4003e62884b12066fc5e60ea358266fe179c0ec5685ba0ace36cafee1c472e8'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='2d